In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
import urllib.request
import os, json, time, copy, re

#### Setting up the connection

In [34]:
driver = webdriver.Chrome()

In [4]:
driver.get('https://www.myntra.com/')

In [3]:
def searchBy(search_string):
    driver.find_element(By.CLASS_NAME, 'desktop-searchBar').send_keys(search_string)
    driver.find_element(By.CLASS_NAME, 'desktop-submit').click()

In [4]:
def getProductLinks():
    while(True):
        if len(product_links) > 2000:
            break

        time.sleep(5)

        for product in driver.find_elements(By.CLASS_NAME, 'product-base'):
            product_links.append(product.find_element(By.XPATH, './a').get_attribute("href")) 

        try:
            driver.find_element(By.CLASS_NAME,'pagination-next').click()
        except:
            driver.close()
            driver.quit()

In [5]:
product_links_all = []

#### Getting each product link

In [6]:
product_links = []

In [13]:
# Search for a category
categories = ['men', 'women']
for cat in categories:
    searchBy(cat) 

    # Get all product links of a particular category
    getProductLinks()
    
    # Add all links to final list
    product_links_all.extend(product_links)
    product_links = []

In [7]:
# Save
# with open('product_links_all.txt', 'w') as f:
#     f.write(str(product_links_all))

# Load
product_links_all = []
x = pd.read_csv('product_links_all.txt', header=None, sep=',').transpose()
for i in range(len(x)):
    product_links_all.append(x.iloc[i, 0][2:-1])

In [8]:
product_links_all[0:5]

['https://www.myntra.com/casual-shoes/skechers/skechers-men-black-equalizer---double-play-slip-on-sneakers/18906834/buy',
 'https://www.myntra.com/flip-flops/beonza/beonza-unisex-sliders-flip-flops/23442272/buy',
 'https://www.myntra.com/headphones/boat/boat-airdopes-black-163-m-with-asap-charge-in-the-ear-bluetooth-headset/19597724/buy',
 'https://www.myntra.com/headphones/oneplus/oneplus-nord-buds-2r-true-wireless-in-ear-earbuds-with-mic/23738230/buy',
 'https://www.myntra.com/headphones/noise/noise-buds-aero-truly-wireless-earbuds-with-45hrs-playtime-and-13mm-driver/23241666/buy']

In [13]:
len(product_links_all)

4100

In [9]:
metadata = {}

In [10]:
products = []

#### Traversing through each link

In [11]:
def traverseLink(start=0, end=1):
    for url in product_links_all[start:end]:
        driver.get(url)

        try:
            metadata['company'] = driver.find_element(By.CLASS_NAME, 'pdp-title').text
        except:
            metadata['company'] = 'None'
        try:
            metadata['name'] = driver.find_element(By.CLASS_NAME, 'pdp-name').text
        except:
            metadata['name'] = 'None'
        try:
            metadata['price'] = driver.find_element(By.CLASS_NAME, 'pdp-price').find_element(By.XPATH, './strong').text
        except:
            metadata['price'] = 'None'
        try:
            metadata['discount'] = driver.find_element(By.CLASS_NAME, 'pdp-discount').text
        except:
            metadata['discount'] = 'None'
        try:
            metadata['specs'] = driver.find_element(By.CLASS_NAME, 'pdp-product-description-content').text
        except:
            metadata['specs'] = 'None'
        try:
            metadata['material'] = driver.find_element(By.CLASS_NAME, 'pdp-sizeFitDescContent').text
        except:
            metadata['material'] = 'None'
        try:
            metadata['rating'] = driver.find_element(By.CLASS_NAME, 'index-overallRating').text
        except:
            metadata['rating'] = 'None'
        try:
            metadata['numBuyers'] = driver.find_element(By.CLASS_NAME, 'index-countDesc').text
        except:
            metadata['numBuyers'] = 'None'
        
        products.append(copy.deepcopy(metadata))

In [12]:
# Traversing through a single link

traverseLink(0, 1)
products[0]

{'company': 'Skechers',
 'name': 'Men Black Equalizer - Double Play Slip-On Sneakers',
 'price': '₹5249',
 'discount': '(25% OFF)',
 'specs': 'Special Technology:\nSkechers Memory Foam Gel-Infused insole\nFeatures:\nA pair of round toe black slip-on sneakers ,has regular styling,\nSlip-on detail\nMesh upper\nCushioned footbed\nShock-absorbing FlexSole lightweight midsole\nFlexible rubber traction outsole\nTextured and patterned outsole\nWarranty: 3 months\nWarranty provided by Brand/Manufacturer',
 'material': 'Mesh\nWipe with clean and dry cloth to remove dust',
 'rating': '4.4\n|\n145 Ratings',
 'numBuyers': '145 Verified Buyers'}

In [35]:
# Complete traversal

traverseLink(2953, -1)
# traverseLink(2500, 3500)

In [36]:
len(products)

4099

In [38]:
json.dump(products, open("products.json", "w"), indent = 4)